## Interpreting your Predictions

In [3]:
# Classification using MobileNetV2
import requests
import tensorflow as tf

import gradio as gr

inception_net = tf.keras.applications.MobileNetV2()  # load the model

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")


def classify_image(inp):
    inp = inp.reshape((-1, 224, 224, 3))
    inp = tf.keras.applications.mobilenet_v2.preprocess_input(inp)
    prediction = inception_net.predict(inp).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}


image = gr.Image(shape=(224, 224))
label = gr.Label(num_top_classes=3)

demo = gr.Interface(
    fn=classify_image, inputs=image, outputs=label, interpretation="default"
)

demo.launch()


2022-10-06 14:19:39.302591: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-06 14:19:39.377064: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-06 14:19:39.380031: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 14:19:39.380038: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

14536120/14536120 [==============================] - 1s 0us/step
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f74724e36a0>, 'http://127.0.0.1:7860/', None)

1/1 [==============================] - 0s 443ms/step


In [1]:
# sex prediction
import gradio as gr

male_words, female_words = ["he", "his", "him"], ["she", "hers", "her"]


def gender_of_sentence(sentence):
    male_count = len([word for word in sentence.split() if word.lower() in male_words])
    female_count = len(
        [word for word in sentence.split() if word.lower() in female_words]
    )
    total = max(male_count + female_count, 1)
    return {"male": male_count / total, "female": female_count / total}

# default interpretation
demo = gr.Interface(
    fn=gender_of_sentence,
    inputs=gr.Textbox(value="She went to his house to get her keys."),
    outputs="label",
    interpretation="default",
)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fa5d17f91c0>, 'http://127.0.0.1:7860/', None)

In [1]:
import re

import gradio as gr

male_words, female_words = ["he", "his", "him"], ["she", "hers", "her"]


def gender_of_sentence(sentence):
    male_count = len([word for word in sentence.split() if word.lower() in male_words])
    female_count = len(
        [word for word in sentence.split() if word.lower() in female_words]
    )
    total = max(male_count + female_count, 1)
    return {"male": male_count / total, "female": female_count / total}


# Number of arguments to interpretation function must
# match number of inputs to prediction function
def interpret_gender(sentence):
    result = gender_of_sentence(sentence)
    is_male = result["male"] > result["female"]
    interpretation = []
    for word in re.split("( )", sentence):
        score = 0
        token = word.lower()
        if (is_male and token in male_words) or (not is_male and token in female_words):
            score = 1
        elif (is_male and token in female_words) or (
            not is_male and token in male_words
        ):
            score = -1
        interpretation.append((word, score))
    # Output must be a list of lists containing the same number of elements as inputs
    # Each element corresponds to the interpretation scores for the given input
    return [interpretation]

# custom interpretation using interpret_gender() function
demo = gr.Interface(
    fn=gender_of_sentence,
    inputs=gr.Textbox(value="She went to his house to get her keys."),
    outputs="label",
    interpretation=interpret_gender,
)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fded3f7f250>, 'http://127.0.0.1:7860/', None)

## Custom Styling

If you'd like to have more fine-grained control over any aspect of your demo, you can also write your own css or pass in a filepath to a css file, with the css parameter of the Interface class.

gr.Interface(..., css="body {background-color: red}")

If you'd like to reference external files in your css, preface the file path (which can be a relative or absolute path) with "file=", for example:

gr.Interface(..., css="body {background-image: url('file=clouds.jpg')}")


## Loading Hugging Face Models and Spaces

Gradio integrates nicely with the Hugging Face Hub, allowing you to load models and Spaces with just one line of code. To use this, simply use the load() method in the Interface class. So:

To load any model from the Hugging Face Hub and create an interface around it, you pass "model/" or "huggingface/" followed by the model name, like these examples:



In [2]:
# gr.Interface.load("huggingface/EleutherAI/gpt-j-6B", 
#     inputs=gr.Textbox(lines=5, label="Input Text")  # customizes the input component
# ).launch()

To load any Space from the Hugging Face Hub and recreate it locally (so that you can customize the inputs and outputs for example), you pass "spaces/" followed by the model name:

In [3]:
# gr.Interface.load("spaces/eugenesiow/remove-bg", inputs="webcam", 
#                   title="Remove your webcam background!").launch()

One of the great things about loading Hugging Face models or spaces using Gradio is that you can then immediately use the resulting Interface object just like function in your Python code (this works for every type of model/space: text, images, audio, video, and even multimodal models):

In [4]:
# io = gr.Interface.load("models/EleutherAI/gpt-neo-2.7B")
# io("It was the best of times")  # outputs model completion

In [6]:
# example for HuggingFace
import gradio as gr
gr.Interface.load("huggingface/gpt2").launch();

Fetching model from: https://huggingface.co/gpt2
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Putting Interfaces in Parallel and Series

**Gradio also lets you mix interfaces very easily using the gradio.Parallel and gradio.Series classes. Parallel lets you put two similar models (if they have the same input type) in parallel to compare model predictions:**

In [2]:
# parallel
import gradio as gr
generator1 = gr.Interface.load("huggingface/gpt2")
generator2 = gr.Interface.load("huggingface/EleutherAI/gpt-neo-2.7B")
generator3 = gr.Interface.load("huggingface/EleutherAI/gpt-j-6B")
gr.Parallel(generator1, generator2, generator3).launch()

Fetching model from: https://huggingface.co/gpt2
Fetching model from: https://huggingface.co/EleutherAI/gpt-neo-2.7B
Fetching model from: https://huggingface.co/EleutherAI/gpt-j-6B

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f93201631f0>, 'http://127.0.0.1:7860/', None)

**Series lets you put models and spaces in series, piping the output of one model into the input of the next model.**

**This demo generates text, then translates it to German, and outputs the final result**

In [1]:
import gradio as gr

generator = gr.Interface.load("huggingface/gpt2")
translator = gr.Interface.load("huggingface/t5-small")
gr.Series(generator, translator).launch()

Fetching model from: https://huggingface.co/gpt2
Fetching model from: https://huggingface.co/t5-small
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f7a2305aa00>, 'http://127.0.0.1:7860/', None)